In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client

c = Client()
c

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:54122,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:54135,Total threads: 2
Dashboard: http://127.0.0.1:54136/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:54124,


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt

from py_clone_detective.clone_counters import LazyCloneCounter
from py_clone_detective.utils import generate_random_cmap

In [ ]:
exp = LazyCloneCounter(
    exp_name="Marcm2a_E7F1",
    img_name_regex=r"a\dg\d\dp\d",
    pixel_size=0.275,
    tot_seg_ch="C0",
)

exp.add_images(
    C0="data/MARCM_experiment/images/C0/*.tif",
    C1="data/MARCM_experiment/images/C1/*.tif",
    C2="data/MARCM_experiment/images/C2/*.tif",
    C3="data/MARCM_experiment/images/C3/*.tif",
)

exp.add_segmentations(C0="data/MARCM_experiment/segmentations/C0_stardist_segs/*.tif")

In [ ]:
exp.make_measurements()

In [ ]:
exp.results_measurements = exp.results_measurements.eval(
    "total_intensity = mean_intensity * area"
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=['int_img_ch == "C1" & seg_ch == "C0" & mean_intensity > 1000'],
    thresh_name="GFP",
    calc_clones=True,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=[
        'int_img_ch == "C2" & seg_ch == "C0" & total_intensity > 5e5',
        'int_img_ch == "C3" & seg_ch == "C0" & mean_intensity < 5000',
    ],
    thresh_name="EC",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=[
        'int_img_ch == "C2" & seg_ch == "C0" & total_intensity < 5e5',
        'int_img_ch == "C3" & seg_ch == "C0" & mean_intensity > 5000',
    ],
    thresh_name="EE",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=["not EC_pos & not EE_pos & area_um2 > 50"],
    thresh_name="preEC",
    calc_clones=False,
)

In [ ]:
exp.add_clones_and_neighbouring_labels(
    thresholds=["not EC_pos & not EE_pos & area_um2 < 50"],
    thresh_name="ISCorEB",
    calc_clones=False,
)

In [ ]:
exp.measure_clones_and_neighbouring_labels(thresh_name="GFP")
exp.measure_clones_and_neighbouring_labels(thresh_name="EC")
exp.measure_clones_and_neighbouring_labels(thresh_name="EE")
exp.measure_clones_and_neighbouring_labels(thresh_name="preEC")
exp.measure_clones_and_neighbouring_labels(thresh_name="ISCorEB")

In [ ]:
df = exp.combine_neighbour_counts_and_measurements().copy()

In [ ]:
df.to_csv('data/example_results.csv')

In [ ]:
df

,int_img,label,GFP_clone,GFPneg_nc,GFPpos_nc,tot_nc,ECneg_nc,ECpos_nc,EEneg_nc,EEpos_nc,...,mean_intensity,centroid-0,centroid-1,area_um2,total_intensity,GFP_pos,EC_pos,EE_pos,preEC_pos,ISCorEB_pos
0,a1g01p1,11,0,2,2,4,1,3,4,0,...,2266.725490,44.054696,89.552116,73.280625,2196457.0,False,True,False,False,False
1,a1g01p1,11,0,2,2,4,1,3,4,0,...,54.444788,44.054696,89.552116,73.280625,52757.0,False,True,False,False,False
2,a1g01p1,11,0,2,2,4,1,3,4,0,...,2335.034056,44.054696,89.552116,73.280625,2262648.0,False,True,False,False,False
3,a1g01p1,11,0,2,2,4,1,3,4,0,...,69.485036,44.054696,89.552116,73.280625,67331.0,False,True,False,False,False
4,a1g01p1,12,0,4,0,4,1,3,4,0,...,3152.119540,53.590038,486.798467,98.690625,4113516.0,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73863,a2g13p3,285,0,3,0,3,2,1,3,0,...,104.699491,771.521222,475.609508,44.543125,61668.0,False,True,False,False,False
73864,a2g13p3,286,0,3,0,3,2,1,3,0,...,1171.554455,770.996700,673.689769,22.914375,354981.0,False,False,False,False,True
73865,a2g13p3,286,0,3,0,3,2,1,3,0,...,32.201320,770.996700,673.689769,22.914375,9757.0,False,False,False,False,True
73866,a2g13p3,286,0,3,0,3,2,1,3,0,...,740.323432,770.996700,673.689769,22.914375,224318.0,False,False,False,False,True
